In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('./others/Dataset-cs-lead/comstock/comtock_merged_15_Min_small.csv')
y_batch = pd.read_csv('./others/Dataset-cs-lead/comstock/comtock_15min_labels.csv')
print(df.shape)
print(df)

(672, 1002)
     Unnamed: 0            Timestamp    73206-0   84687-0    148366-0  \
0             0  2018-01-01 00:15:00  12.546527  0.831156   98.539537   
1             1  2018-01-01 00:30:00  12.673777  0.822060  104.659842   
2             2  2018-01-01 00:45:00  12.798760  0.790814  104.603952   
3             3  2018-01-01 01:00:00  12.960590  0.759746   96.407704   
4             4  2018-01-01 01:15:00  13.193791  0.727807  103.599638   
..          ...                  ...        ...       ...         ...   
667         667  2018-01-07 23:00:00   7.695500  0.557569   80.922287   
668         668  2018-01-07 23:15:00   7.765408  0.557728   81.345516   
669         669  2018-01-07 23:30:00   7.840582  0.557886   79.945413   
670         670  2018-01-07 23:45:00   7.911101  0.558031   82.332683   
671         671  2018-01-08 00:00:00   7.977462  0.558181   89.367624   

     126539-0    23028-0   116299-0  122105-0   16125-0  ...   34613-0  \
0    0.956724   8.133667  52.922663  

In [4]:
y_batch.describe()

,cooling_ON,exterior_lighting_ON,fans_ON,heat_recovery_ON,heat_rejection_ON,heating_ON,interior_equipment_ON,interior_lighting_ON,pumps_ON,refrigeration_ON,water_systems_ON,id
count,1000.000000,1000.0,1000.000000,1000.0,1000.000000,1000.000000,1000.0,1000.0,1000.0,1000.000000,1000.000000,1000.000000
mean,0.764000,1.0,0.902000,0.0,0.117000,0.449000,1.0,1.0,1.0,0.316000,0.726000,70763.824000
std,0.424835,0.0,0.297463,0.0,0.321581,0.497641,0.0,0.0,0.0,0.465146,0.446232,40650.888331
min,0.000000,1.0,0.000000,0.0,0.000000,0.000000,1.0,1.0,1.0,0.000000,0.000000,93.000000
25%,1.000000,1.0,1.000000,0.0,0.000000,0.000000,1.0,1.0,1.0,0.000000,0.000000,34882.250000
50%,1.000000,1.0,1.000000,0.0,0.000000,0.000000,1.0,1.0,1.0,0.000000,1.000000,70554.000000
75%,1.000000,1.0,1.000000,0.0,0.000000,1.000000,1.0,1.0,1.0,1.000000,1.000000,104219.000000
max,1.000000,1.0,1.000000,0.0,1.000000,1.000000,1.0,1.0,1.0,1.000000,1.000000,148985.000000


In [ ]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from transformers import AutoConfig
from tsfm_public.models.tspulse.modeling_tspulse import TSPulseForClassification
from collections import OrderedDict
from safetensors.torch import load_file
import os
import logging
from tqdm.auto import tqdm
import torch.nn as nn
from sklearn.metrics import accuracy_score, f1_score, classification_report


In [6]:
X= df.drop(columns= ["Unnamed: 0", "Timestamp"]).T.values
y = y_batch["cooling_ON"].values
print(X)
print(X.shape)
print(y)
print(y.shape)

[[ 12.54652662  12.67377669  12.79875963 ...   7.84058213   7.91110101
    7.97746241]
 [  0.83115624   0.82206018   0.79081381 ...   0.55788569   0.55803061
    0.55818051]
 [ 98.53953711 104.65984218 104.60395193 ...  79.94541343  82.33268299
   89.36762355]
 ...
 [  2.89782042   2.80151787   2.71719464 ...   1.73580449   1.74226426
    1.74556636]
 [  0.8590106    0.85959741   0.86043797 ...   1.02703109   1.0122141
    0.99233404]
 [  1.09697354   1.10985971   1.12343835 ...   0.75446774   0.76028772
    0.76547748]]
(1000, 672)
[1 1 1 0 0 1 1 1 0 1 0 1 1 0 1 1 1 1 1 1 1 1 0 1 1 1 0 1 0 1 1 0 1 1 0 1 1
 1 1 1 0 1 1 1 1 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 0 1 0 1 1 1 1
 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 0 1 0 1 1 0 0 1 1 1 0 1 1 0 1 1 0 1 0 1 1 1
 1 1 1 1 0 0 0 1 0 1 1 1 1 0 0 1 0 0 1 1 1 1 1 1 0 1 1 1 0 0 1 1 1 1 1 0 1
 1 1 0 1 1 1 1 1 0 0 1 1 0 1 1 0 1 1 1 1 0 1 0 1 0 1 1 1 1 1 1 1 1 0 1 0 0
 0 1 0 1 1 0 1 1 1 1 0 1 1 1 1 0 1 0 0 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1
 1 0 1 1 1 

In [7]:
# from sktime.classification.kernel_based import Arsenal
# from sktime.datasets import load_unit_test

# clf = Arsenal(num_kernels=100, n_estimators=100)
# clf.fit_predict(X, y)
# clf.score(X, y)

In [8]:

def load_tspulse_model(ckpt: str, model_class, config=None): # Removed num_labels, as it's in config
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    if config is None:
        config = AutoConfig.from_pretrained(ckpt, trust_remote_code=True)
    
    st_path = os.path.join(ckpt, "model.safetensors")
    bin_path = os.path.join(ckpt, "pytorch_model.bin")
    
    state_dict = None
    if os.path.exists(st_path):
        state_dict = load_file(st_path, device="cpu")
    elif os.path.exists(bin_path):
        state_dict = torch.load(bin_path, map_location="cpu")
    
    if state_dict is not None:
        new_sd = OrderedDict()
        for k, v in state_dict.items():
            nk = k.replace("_orig_mod.", "", 1) if k.startswith("_orig_mod.") else k
            new_sd[nk] = v
        model = model_class(config).to(device)
        model.load_state_dict(new_sd, strict=False)
    else:
        raise FileNotFoundError(f"No model file found in checkpoint directory: {ckpt}")
    
    return model


In [9]:

logging.basicConfig(level=logging.INFO, format='INFO:%(name)s:%(message)s')
logger = logging.getLogger(__name__)

class     ClassificationDataset(Dataset):
    def __init__(self, X: np.ndarray, y: np.ndarray):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return {
            "past_values": self.X[idx],
            "target_values": self.y[idx], 
        }


In [ ]:

def evaluate_model(model, dataloader, device, X_test_raw, label_name, num_labels):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating"):
            inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels'}
            labels = batch['target_values'].to(device)
            
            inputs['past_observed_mask'] = (inputs['past_values'] != 0.0)
            outputs = model(**inputs) 

            logits = outputs.prediction_outputs
            
            if logits.dim() > 2:
                logits = logits.mean(dim=1)
            if labels.dim() > 1:
                labels = labels[:, 0]
            labels = labels.view(-1)
            # print(labels)
            # print(logits)
            # predictions2 = torch.argmax(logits, dim=-1)
            predictions = (torch.sigmoid(logits) > 0.5).int().view(-1)
            # print(predictions)
            # print(predictions2)
            all_preds.extend(predictions.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            
    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)

    accuracy = accuracy_score(all_labels, all_preds)
    f1_macro = f1_score(all_labels, all_preds, average='macro', zero_division=0)
    
    if np.any(all_labels == 1):
        true_mean_label_1 = X_test_raw[all_labels == 1].mean()
    else:
        true_mean_label_1 = np.nan

    if np.any(all_preds == 1):
        predicted_mean_label_1 = X_test_raw[all_preds == 1].mean()
    else:
        predicted_mean_label_1 = np.nan

    predicted_positives_count = np.sum(all_preds == 1)

    print("\n" + "="*40)
    print(f"Classification Report for: {label_name}")
    print("="*40)

    all_possible_labels = list(range(num_labels))
    target_names = [f"class_{i}" for i in all_possible_labels]

    report = classification_report(
        all_labels,
        all_preds,
        labels=all_possible_labels,
        target_names=target_names,
        zero_division=0
    )
    print(report)
    print("="*40 + "\n")

    return { "accuracy": accuracy, "f1_macro": f1_macro, "true_mean_label_1": true_mean_label_1, "predicted_mean_label_1": predicted_mean_label_1, "predicted_positives_count": predicted_positives_count }



In [30]:

# --- Data Loading and Preprocessing ---
# df = pd.read_csv('/home/samy/Abhinav/implementations/Dataset/comstock/comtock_merged_15_Min_small.csv')
# labels_df = pd.read_csv('/home/samy/Abhinav/implementations/Dataset/comstock/comtock_15min_labels.csv')
df = pd.read_csv('./others/Dataset-cs-lead/comstock/comtock_merged_15_Min_small.csv')
labels_df = pd.read_csv('./others/Dataset-cs-lead/comstock/comtock_15min_labels.csv')
X_raw = df.drop(columns=["Unnamed: 0", "Timestamp"]).T.values
context_length = 512
num_series, series_length = X_raw.shape
X_processed = np.zeros((num_series, context_length))

for i in range(num_series):
    series = X_raw[i, :]
    series = np.nan_to_num(series, nan=0.0)
    if len(series) > context_length:
        X_processed[i, :] = series[:context_length]
    else:
        X_processed[i, :len(series)] = series
X_processed = X_processed[:, :, np.newaxis]



In [31]:
X_processed.shape

(1000, 512, 1)

In [32]:
model = TSPulseForClassification.from_pretrained(
    "ibm-granite/granite-timeseries-tspulse-r1",
    revision="tspulse-block-dualhead-512-p16-r1",
    num_targets = 1

)

INFO:p-3992:t-25704:modeling_tspulse.py:_init_weights:Initializing Linear layers with method: pytorch
INFO:p-3992:t-25704:modeling_tspulse.py:_init_weights:Initializing Linear layers with method: pytorch
Some weights of TSPulseForClassification were not initialized from the model checkpoint at ibm-granite/granite-timeseries-tspulse-r1 and are newly initialized: ['decoder_with_head.head.head_norm.norm.bias', 'decoder_with_head.head.head_norm.norm.weight', 'decoder_with_head.head.loc_scale_norm.bias', 'decoder_with_head.head.loc_scale_norm.weight', 'decoder_with_head.head.projection.bias', 'decoder_with_head.head.projection.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [33]:
model(torch.tensor(X_processed[:2].astype(np.float32), dtype=torch.float32,device='cpu'),return_loss =False)

TSPulseForClassificationOutput(loss=tensor([0.]), prediction_outputs=tensor([[-0.4270],
        [-1.0841]], grad_fn=<AddmmBackward0>), backbone_hidden_state=tensor([[[ 1.1214e-01, -2.0139e-02,  2.3891e-02,  ...,  1.5239e+00,
           4.0007e+00,  8.9007e-01]],

        [[ 2.3946e-02, -1.4718e-02,  1.8837e-03,  ...,  1.6734e+00,
           4.0285e+00,  9.0510e-01]]], grad_fn=<ViewBackward0>), decoder_hidden_state=tensor([[[ 0.0834, -0.0150,  0.0083,  ...,  1.5113,  3.9480,  0.9065]],

        [[-0.0550,  0.0048, -0.0165,  ...,  1.6804,  4.0494,  0.9916]]],
       grad_fn=<ViewBackward0>), loc=tensor([[[8.6741]],

        [[0.6890]]], grad_fn=<DivBackward0>), scale=tensor([[[1.2573]],

        [[0.1512]]], grad_fn=<SqrtBackward0>))

## Zero Shot

In [34]:
# --- Main Training Loop ---
target_labels = ["cooling_ON","fans_ON", "heating_ON", "refrigeration_ON", "water_systems_ON"]
pretrained_model_path = "./check_points_b/TTM_cl-512_fl-96_pl-64_apl-3_ne-25_es-True/tspulse_dual_domain_pretrained"
base_output_dir = "./finetuned_output_1000_for_classification"
os.makedirs(base_output_dir, exist_ok=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
all_results = {}


In [ ]:
for label_name in target_labels:
    logger.info("="*80)
    logger.info(f"Starting fine-tuning for label: {label_name}")
    
    y_current = labels_df[label_name].values
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y_current)
    num_labels = len(np.unique(y_encoded))
    
    logger.info(f"Label '{label_name}' has {num_labels} unique classes: {np.unique(y_encoded)}")
    if num_labels <= 1:
        logger.warning(f"Skipping label '{label_name}' as it has only one class.")
        continue

    X_train, X_test, y_train, y_test = train_test_split(
        X_processed, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
    )
    
    # train_dataset = ClassificationDataset(X_train, y_train)
    test_dataset = ClassificationDataset(X_test, y_test)
    X_test_raw_for_eval = X_test.squeeze()
    # train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)
    
    y_train_flat = y_train.ravel()
    present_classes = np.unique(y_train_flat)
    # final_weights_list = [1.0] * num_labels # Default weight
    if len(present_classes) > 1:
        class_counts = np.bincount(y_train_flat)
        n_train_samples = len(y_train_flat)
        num_present_classes = len(present_classes)
        for class_idx in present_classes:
            # Check if class_idx is within the bounds of class_counts
            if class_idx < len(class_counts):
                weight = n_train_samples / (num_present_classes * class_counts[class_idx] + 1e-6)
                # final_weights_list[class_idx] = weight
    
    # logger.info(f"Final weight list for all {num_labels} classes: {final_weights_list}")

    # Configure the model
    config = AutoConfig.from_pretrained(pretrained_model_path, trust_remote_code=True)

    model_finetune = model #load_tspulse_model(ckpt=pretrained_model_path, model_class=TSPulseForClassification, config=config)
    model_finetune.to(device)

    final_test_results = evaluate_model(model_finetune, test_dataloader, device, X_test_raw_for_eval, label_name, num_labels)
    all_results[label_name] = final_test_results
    final_model_path = os.path.join(base_output_dir, f"tspulse_finetuned_{label_name}")
    # model_finetune.save_pretrained(final_model_path)
    print(f"Model for '{label_name}' saved to: {final_model_path}\n\n")

# --- Final Summary ---
logger.info("="*80)
logger.info("Final Classification Summary for All Labels")
logger.info("="*80)
results_df = pd.DataFrame(all_results).T
print(results_df)


summary_filename = "classification_zeroshot_comstock_summary.txt"
summary_string = results_df.to_string()

try:
    with open(summary_filename, "w") as f:
        f.write("Final Classification Summary for All Labels\n")
        f.write("============================================\n\n")
        f.write(summary_string)
    logger.info(f"Final summary successfully saved to: {summary_filename}")
except Exception as e:
    logger.error(f"Failed to save summary file. Error: {e}")

INFO:p-3992:t-25704:4195262686.py:<module>:================================================================================
INFO:p-3992:t-25704:4195262686.py:<module>:Starting fine-tuning for label: cooling_ON
INFO:p-3992:t-25704:4195262686.py:<module>:Label 'cooling_ON' has 2 unique classes: [0 1]
INFO:p-3992:t-25704:4195262686.py:<module>:Final weight list for all 2 classes: [np.float64(2.116402110803169), np.float64(0.6546644839159864)]


Evaluating:   0%|          | 0/7 [00:00<?, ?it/s]

c:\Users\srina\anaconda3\envs\tspulse_env\lib\site-packages\torch\nn\modules\loss.py:616: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
c:\Users\srina\anaconda3\envs\tspulse_env\lib\site-packages\torch\nn\modules\loss.py:616: UserWarning: Using a target size (torch.Size([8])) that is different to the input size (torch.Size([8, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
INFO:p-3992:t-25704:4195262686.py:<module>:================================================================================
INFO:p-3992:t-25704:4195262686.py:<module>:Starting fine-tuning for label: fans_ON
INFO:p-3992:t-25704:4195262686.py:<module>:Label 'fans_ON' has 2 


Classification Report for: cooling_ON
              precision    recall  f1-score   support

     class_0       0.23      1.00      0.38        47
     class_1       0.00      0.00      0.00       153

    accuracy                           0.23       200
   macro avg       0.12      0.50      0.19       200
weighted avg       0.06      0.23      0.09       200


Model for 'cooling_ON' saved to: ./finetuned_output_1000_for_classification\tspulse_finetuned_cooling_ON




Evaluating:   0%|          | 0/7 [00:00<?, ?it/s]

...

c:\Users\srina\anaconda3\envs\tspulse_env\lib\site-packages\torch\nn\modules\loss.py:616: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
c:\Users\srina\anaconda3\envs\tspulse_env\lib\site-packages\torch\nn\modules\loss.py:616: UserWarning: Using a target size (torch.Size([8])) that is different to the input size (torch.Size([8, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
INFO:p-3992:t-25704:4195262686.py:<module>:================================================================================
INFO:p-3992:t-25704:4195262686.py:<module>:Starting fine-tuning for label: water_systems_ON
INFO:p-3992:t-25704:4195262686.py:<module>:Label 'water_


Classification Report for: refrigeration_ON
              precision    recall  f1-score   support

     class_0       0.69      1.00      0.81       137
     class_1       0.00      0.00      0.00        63

    accuracy                           0.69       200
   macro avg       0.34      0.50      0.41       200
weighted avg       0.47      0.69      0.56       200


Model for 'refrigeration_ON' saved to: ./finetuned_output_1000_for_classification\tspulse_finetuned_refrigeration_ON




Evaluating:   0%|          | 0/7 [00:00<?, ?it/s]

c:\Users\srina\anaconda3\envs\tspulse_env\lib\site-packages\torch\nn\modules\loss.py:616: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
c:\Users\srina\anaconda3\envs\tspulse_env\lib\site-packages\torch\nn\modules\loss.py:616: UserWarning: Using a target size (torch.Size([8])) that is different to the input size (torch.Size([8, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
INFO:p-3992:t-25704:4195262686.py:<module>:================================================================================
INFO:p-3992:t-25704:4195262686.py:<module>:Final Classification Summary for All Labels
INFO:p-3992:t-25704:4195262686.py:<module>:==================


Classification Report for: water_systems_ON
              precision    recall  f1-score   support

     class_0       0.28      1.00      0.43        55
     class_1       0.00      0.00      0.00       145

    accuracy                           0.28       200
   macro avg       0.14      0.50      0.22       200
weighted avg       0.08      0.28      0.12       200


Model for 'water_systems_ON' saved to: ./finetuned_output_1000_for_classification\tspulse_finetuned_water_systems_ON


                  accuracy  f1_macro  true_mean_label_1  \
cooling_ON           0.235  0.190283          22.682408   
fans_ON              0.100  0.090909          23.515069   
heating_ON           0.550  0.354839          27.918585   
refrigeration_ON     0.685  0.406528          44.345345   
water_systems_ON     0.275  0.215686          20.123054   

                  predicted_mean_label_1  predicted_positives_count  
cooling_ON                           NaN                        0.0  
fans_ON      

## Fine Tuning